In [43]:
import numpy as np
from numba import types, njit,int64,jit,cuda,prange,vectorize
from BPnumba.DABC import createDABC
from BPnumba.EDABC import createEDABC,BeeSearch
from BPnumba.BoxN import create_ItemBin,ItemBin,InstaceBoxes
from BPnumba.Poblation import CreateRotHPob,EvalPob
import matplotlib.pyplot as plt
from BPmodule.BPGen import GetInstance
from numba.typed import List as NumbaList
import random as rd
import numba as nb
import random

In [44]:
def zerovec(y, x): return np.zeros(shape=[x, y], dtype=np.float64)
createText = lambda x,y : np.savetxt(
    fname = "Results/"+x + ".csv",
    X= y,
    delimiter=" ",
    fmt='%g')

In [45]:
@njit#(parallel=True)
def BenchABC(i,boxes,bin,lst,lst2,lst3,maxItr,pop,rot):
    #fi = NumbaList(np.zeros(maxItr,dtype=np.float64))
    UBOXES = InstaceBoxes(boxes)
    pob = CreateRotHPob(pop =pop, ITEMS = UBOXES, rotType=rot)
    EvalPob(pob= pob,boxData=UBOXES,container=bin,rotation=rot)
    
    ind = BeeSearch(
    mutType=0,
    numItr=maxItr,
    m_sites=4,
    elite_sites=2,
    elite_bees=4,
    nonelite_bees=2,
    ColonyWorker=pob.copy(),
    datos=UBOXES.copy(),
    contenedor=bin,
    rot=rot)
    
    ind2 = BeeSearch(
    mutType=1,
    numItr=maxItr,
    m_sites=4,
    elite_sites=2,
    elite_bees=4,
    nonelite_bees=2,
    ColonyWorker=pob.copy(),
    datos=UBOXES.copy(),
    contenedor=bin,
    rot=rot)
    
    ind3 = BeeSearch(
    mutType=2,
    numItr=maxItr,
    m_sites=4,
    elite_sites=2,
    elite_bees=4,
    nonelite_bees=2,
    ColonyWorker=pob.copy(),
    datos=UBOXES.copy(),
    contenedor=bin,
    rot=rot)

    lst[i]=ind.fi
    lst2[i] = ind2.fi
    lst3[i] = ind3.fi

In [46]:
@njit(parallel=True)
def BABC(rango,rango2,contenedor,data:list[list[list[int]]],agL,agL2,agL3,maxItr,numpop,rotation):
    for p in prange(rango,rango2):
        BenchABC(
                i=p,
                boxes=data[p], 
                bin=contenedor,
                maxItr= maxItr,
                lst= agL,
                lst2=agL2,
                lst3=agL3,
                pop=numpop,
                rot=rotation)

In [58]:
def Bench(nm,r):
    BR1 = GetInstance(nm)
    agL = zerovec(1, 100)
    agL2 = zerovec(1, 100)
    agL3 = zerovec(1, 100)
    for th in np.arange(0,r,4):
        BABC(rango=th,rango2=th+4,contenedor=BR1[0],data=BR1[1],agL=agL,agL2=agL2,agL3=agL3,maxItr=1000,numpop=20,rotation=6)
        createText(nm+"/ABC/"+"ABC"+str(0),agL)
        createText(nm+"/ABC/"+"ABC"+str(1),agL2)
        createText(nm+"/ABC/"+"ABC"+str(2),agL3)

In [54]:
for th in np.arange(0,8,4):
    st = ""
    for t in range(th,th+4):
        st += str(t)
    print(st)
        

0123
4567


In [48]:
r=100
agL = zerovec(1, 100)
agL2 = zerovec(1, 100)
agL3 = zerovec(1, 100)
BR1 = GetInstance("P2A2")
cont = BR1[0]
dataBoxes=BR1[1]
boxes = dataBoxes[0]

In [50]:
BenchABC(
                i=0,
                boxes=boxes, 
                bin=cont,
                maxItr= 1000,
                lst= agL,
                
                lst2=agL2,
                lst3=agL3,
                pop=20,
                rot=6)

In [60]:
Bench("P2A2",100)

In [ ]:
Bench("P3A2",100)

In [19]:
pop=20
maxItr=1000
rot=6
mut=0
m=4 #good sites
e=2  #elite sites
nep=4 #elite bees
nsp=2 #non elite bees


In [26]:
BR1 = GetInstance("P2A2")
cont = BR1[0]
dataBoxes=BR1[1]
boxes = dataBoxes[0]

In [27]:
UBOXES = InstaceBoxes(boxes)
pob = CreateRotHPob(pop =pop, ITEMS = UBOXES, rotType=rot)
EvalPob(pob,UBOXES,cont,rot)

In [28]:
ind = BeeSearch(
    mutType=mut,
    numItr=maxItr,
    m_sites=m,
    elite_sites=e,
    elite_bees=nep,
    nonelite_bees=nsp,
    ColonyWorker=pob.copy(),
    datos=UBOXES.copy(),
    contenedor=cont,
    rot=rot)

In [29]:
ind1 = BeeSearch(
    mutType=1,
    numItr=maxItr,
    m_sites=m,
    elite_sites=e,
    elite_bees=nep,
    nonelite_bees=nsp,
    ColonyWorker=pob.copy(),
    datos=UBOXES.copy(),
    contenedor=cont,
    rot=rot)

In [30]:
ind2 = BeeSearch(
    mutType=2,
    numItr=maxItr,
    m_sites=m,
    elite_sites=e,
    elite_bees=nep,
    nonelite_bees=nsp,
    ColonyWorker=pob.copy(),
    datos=UBOXES.copy(),
    contenedor=cont,
    rot=rot)

In [31]:
ind.fi,ind1.fi,ind2.fi

(0.980944, 0.92704, 0.967488)